In [78]:
import os
import time
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"
from os import listdir
import pandas as pd
import numpy as np
import glob
import cv2
import json
from os.path import expanduser
import splitfolders
import shutil
from define_path import Def_Path
from datetime import datetime

from tqdm import tqdm

import torch 
import torchvision
from torchvision import models
from torchvision.models.detection.rpn import AnchorGenerator
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn 
import torchvision.transforms as T
from torchvision.transforms import functional as F
from torchsummary import summary
from sklearn.model_selection import train_test_split

import albumentations as A # Library for augmentations

import matplotlib.pyplot as plt 
from PIL import Image

import transforms, utils, engine, train
from utils import collate_fn
from engine import train_one_epoch, evaluate

t = torch.cuda.get_device_properties(0).total_memory
print(t)
torch.cuda.empty_cache()

r = torch.cuda.memory_reserved(0)
print(r)
a = torch.cuda.memory_allocated(0)
print(a)
# f = r-a  # free inside reserved

weights_path = '/home/jc-merlab/Pictures/Data/trained_models/keypointsrcnn_weights_sim_b1_e25_v0.pth'

16908615680
719323136
505855488


In [79]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
# torch.cuda.set_per_process_memory_fraction(0.9, 0)
print(device)

cuda


In [80]:
import torch
import torch.nn as nn
import torch.nn.functional as func
import math
import numpy as np
from torch.autograd import Variable
import torch_geometric.nn as pyg
from torch_geometric.data import Data

_EPS = 1e-10

class MLP(nn.Module):
    """Two-layer fully-connected ELU net with batch norm."""

    def __init__(self, n_in, n_hid, n_out, do_prob=0.):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(n_in, n_hid)
        self.fc2 = nn.Linear(n_hid, n_out)
#         self.bn = nn.BatchNorm1d(n_out)
        self.dropout_prob = do_prob

        self.init_weights()

    def init_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Linear):
                nn.init.xavier_normal_(m.weight.data)
                m.bias.data.fill_(0.1)
            elif isinstance(m, nn.BatchNorm1d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()

    def batch_norm(self, inputs):
        x = inputs.view(inputs.size(0) * inputs.size(1), -1)
        x = self.bn(x)
        return x.view(inputs.size(0), inputs.size(1), -1)

#     def forward(self, inputs):
#         print(inputs)
#         # Input shape: [num_sims, num_things, num_features]
#         x = func.elu(self.fc1(inputs))
#         x = func.dropout(x, self.dropout_prob, training=self.training)
#         x = func.elu(self.fc2(x))
#         return self.batch_norm(x)

    def forward(self, inputs):
#         print("Input shape before any operations: ", inputs.shape)

        # Flatten the last two dimensions for the linear layer input
#         x = inputs.view(inputs.size(0), -1)
#         print("Input shape after flattening: ", x.shape)

        x = func.elu(self.fc1(inputs))
        x = func.dropout(x, self.dropout_prob, training=self.training)
        x = func.elu(self.fc2(x))

        # Assuming you want to maintain the second dimension for some reason
        # (like temporal sequence in a RNN), you would reshape the output
        # back to the desired shape. If not, this step is unnecessary.
        # output = x.view(inputs.size(0), inputs.size(1), -1)
        # print("Output shape after forward pass: ", output.shape)

        return x


class GraphEncoder(nn.Module):
    def __init__(self, n_in, n_hid, n_out=4, do_prob=0., factor=False):
        super(GraphEncoder, self).__init__()

        self.factor = factor

        self.mlp1 = MLP(n_in, n_hid, n_hid, do_prob)
        self.mlp2 = MLP(n_hid * 2, n_hid, n_hid, do_prob)
        self.mlp3 = MLP(n_hid, n_hid, n_hid, do_prob)
        if self.factor:
            self.mlp4 = MLP(n_hid * 3, n_hid, n_hid, do_prob)
            print("Using factor graph MLP encoder.")
        else:
            self.mlp4 = MLP(n_hid * 2, n_hid, n_hid, do_prob)
            print("Using MLP graph encoder.")
        self.fc_out = nn.Linear(n_hid, n_out)
        self.init_weights()

    def init_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Linear):
                nn.init.xavier_normal_(m.weight.data)
                m.bias.data.fill_(0.1)

    def edge2node(self, x, rel_rec, rel_send):
        # NOTE: Assumes that we have the same graph across all samples.
        incoming = torch.matmul(rel_rec.t(), x)
        return incoming / incoming.size(1)

    def node2edge(self, x, rel_rec, rel_send):
        # NOTE: Assumes that we have the same graph across all samples.
        receivers = torch.matmul(rel_rec, x)
        senders = torch.matmul(rel_send, x)
        edges = torch.cat([receivers, senders], dim=2)
        return edges

    def forward(self, inputs, rel_rec, rel_send):
        # Input shape: [num_sims, num_atoms, num_timesteps, num_dims]
        x = inputs.view(inputs.size(0), inputs.size(1), -1)
#         print("x shape:", x.shape)
#         print("rel_rec shape:", rel_rec.shape)
#         print("rel_send shape:", rel_send.shape)

        # New shape: [num_sims, num_atoms, num_timesteps*num_dims]
        x = self.mlp1(x)  # 2-layer ELU net per node

        x = self.node2edge(x, rel_rec, rel_send)
        x = self.mlp2(x)
        x_skip = x         
        if self.factor:
            x = self.edge2node(x, rel_rec, rel_send)
            x = self.mlp3(x)
            x = self.node2edge(x, rel_rec, rel_send)
            x = torch.cat((x, x_skip), dim=2)  # Skip connection
            x = self.mlp4(x)
        else:
            x = self.mlp3(x)
            x = torch.cat((x, x_skip), dim=2)  # Skip connection
            x = self.mlp4(x)

        return self.fc_out(x)    
    
class GraphDecoder(nn.Module):

    def __init__(self, n_in_node, edge_types, msg_hid, msg_out, n_hid,
                 do_prob=0., skip_first=False):
        super(GraphDecoder, self).__init__()
        self.msg_fc1 = nn.ModuleList(
            [nn.Linear(2 * n_in_node, msg_hid) for _ in range(edge_types)])
        self.msg_fc2 = nn.ModuleList(
            [nn.Linear(msg_hid, msg_out) for _ in range(edge_types)])
        self.msg_out_shape = msg_out
        self.skip_first_edge_type = skip_first

        self.out_fc1 = nn.Linear(n_in_node + msg_out, n_hid)
        self.out_fc2 = nn.Linear(n_hid, n_hid)
        self.out_fc3 = nn.Linear(n_hid, n_in_node)

        print('Using learned graph decoder.')

        self.dropout_prob = do_prob

    def single_step_forward(self, single_timestep_inputs, rel_rec, rel_send,
                            single_timestep_rel_type):

        # single_timestep_inputs has shape
        # [batch_size, num_timesteps, num_atoms, num_dims]

        # single_timestep_rel_type has shape:
        # [batch_size, num_timesteps, num_atoms*(num_atoms-1), num_edge_types]

        # Node2edge
        receivers = torch.matmul(rel_rec, single_timestep_inputs)
        senders = torch.matmul(rel_send, single_timestep_inputs)
        pre_msg = torch.cat([receivers, senders], dim=-1)

        all_msgs = Variable(torch.zeros(pre_msg.size(0), pre_msg.size(1),self.msg_out_shape))
        if single_timestep_inputs.is_cuda:
            all_msgs = all_msgs.cuda()

        if self.skip_first_edge_type:
            start_idx = 1
        else:
            start_idx = 0

        # Run separate MLP for every edge type
        # NOTE: To exlude one edge type, simply offset range by 1
        for i in range(start_idx, len(self.msg_fc2)):
            msg = func.relu(self.msg_fc1[i](pre_msg))
            msg = func.dropout(msg, p=self.dropout_prob)
            msg = func.relu(self.msg_fc2[i](msg))
            msg = msg * single_timestep_rel_type[:, :, i:i + 1]
            all_msgs += msg

        # Aggregate all msgs to receiver
        agg_msgs = all_msgs.transpose(-2, -1).matmul(rel_rec).transpose(-2, -1)
        agg_msgs = agg_msgs.contiguous()

        # Skip connection
        aug_inputs = torch.cat([single_timestep_inputs, agg_msgs], dim=-1)

        # Output MLP
        pred = func.dropout(func.relu(self.out_fc1(aug_inputs)), p=self.dropout_prob)
        pred = func.dropout(func.relu(self.out_fc2(pred)), p=self.dropout_prob)
        pred = self.out_fc3(pred)
#        print(pred.shape,single_timestep_inputs.shape)

        # Predict position/velocity difference
        return single_timestep_inputs + pred

    def forward(self, inputs, rel_type, rel_rec, rel_send, pred_steps=1):
        # NOTE: Assumes that we have the same graph across all samples.


        # Only take n-th timesteps as starting points (n: pred_steps)
        last_pred = inputs[:, :, :]
        #asa
        curr_rel_type = rel_type[:, :, :]
        preds=[]
        #print(curr_rel_type.shape)
        # NOTE: Assumes rel_type is constant (i.e. same across all time steps).

        # Run n prediction steps
        #for step in range(0, pred_steps):
        last_pred = self.single_step_forward(last_pred, rel_rec, rel_send,
                                                 curr_rel_type)
        preds.append(last_pred)

        sizes = [preds[0].size(0), preds[0].size(1),
                 preds[0].size(2)]

        output = Variable(torch.zeros(sizes))
        if inputs.is_cuda:
            output = output.cuda()

        # Re-assemble correct timeline
        for i in range(len(preds)):
            output[:, :, :] = preds[i]

        pred_all = output[:, :, :]

        # NOTE: We potentially over-predicted (stored in future_pred). Unused.
        # future_pred = output[:, (inputs.size(1) - 1):, :, :]

        return pred_all#.transpose(1, 2).contiguous()    


In [81]:
def my_softmax(input, axis=1):
    trans_input = input.transpose(axis, 0).contiguous()
    soft_max_1d = func.softmax(trans_input,dim=0)
    return soft_max_1d.transpose(axis, 0)


In [82]:
class KeypointPipeline(nn.Module):
    def __init__(self, weights_path):
        super(KeypointPipeline, self).__init__()  
        self.keypoint_model = torch.load(weights_path).to(device)
        self.encoder = GraphEncoder(4,256,4,0.5,False)
#                                     cfg.MODEL.ROI_GRAPH_HEAD.ENCODER_HIDDEN,
#                                     cfg.MODEL.ROI_GRAPH_HEAD.EDGE_TYPES,
#                                     cfg.MODEL.ROI_GRAPH_HEAD.ENCODER_DROPOUT,
#                                     cfg.MODEL.ROI_GRAPH_HEAD.ENCODER_FACTOR)
        self.decoder = GraphDecoder(n_in_node=4,
                                 edge_types=2,
                                 msg_hid=256,
                                 msg_out=256,
                                 n_hid=256,
                                 do_prob=0.5,
                                 skip_first=False)
        
        self.off_diag = np.ones([6,6]) - np.eye(6)

        self.rel_rec = np.array(encode_onehot(np.where(self.off_diag)[1]), dtype=np.float32)
        self.rel_send = np.array(encode_onehot(np.where(self.off_diag)[0]), dtype=np.float32)
        self.rel_rec = torch.FloatTensor(self.rel_rec)
        self.rel_send = torch.FloatTensor(self.rel_send)

        # Define a unidirectional cyclical graph
#         num_nodes = 6
#         self.off_diag = np.zeros([num_nodes, num_nodes])
        
#         # Creating a cycle: 1->2, 2->3, ..., 6->1
#         for i in range(num_nodes):
#             self.off_diag[i, (i + 1) % num_nodes] = 1

#         # Update rel_rec and rel_send based on the new off_diag
#         self.rel_rec = np.array(encode_onehot(np.where(self.off_diag)[1]), dtype=np.float32)
#         self.rel_send = np.array(encode_onehot(np.where(self.off_diag)[0]), dtype=np.float32)
#         self.rel_rec = torch.FloatTensor(self.rel_rec).to(device)
#         self.rel_send = torch.FloatTensor(self.rel_send).to(device)


        self.encoder= self.encoder.cuda()
        self.decoder = self.decoder.cuda()
        self.rel_rec = self.rel_rec.cuda()
        self.rel_send = self.rel_send.cuda()
    
    def process_model_output(self, output):
        scores = output[0]['scores'].detach().cpu().numpy()
        high_scores_idxs = np.where(scores > 0.7)[0].tolist()

        post_nms_idxs = torchvision.ops.nms(output[0]['boxes'][high_scores_idxs], 
                                            output[0]['scores'][high_scores_idxs], 0.3).cpu().numpy()

        confidence = output[0]['scores'][high_scores_idxs][post_nms_idxs].detach().cpu().numpy()
        labels = output[0]['labels'][high_scores_idxs][post_nms_idxs].detach().cpu().numpy()
        keypoints = []
        for idx, kps in enumerate(output[0]['keypoints'][high_scores_idxs][post_nms_idxs].detach().cpu().numpy()):
            keypoints.append(list(map(int, kps[0,0:2])) + [confidence[idx]] + [labels[idx]])
        
        # Sort keypoints based on label
        keypoints.sort(key=lambda x: x[-1])
        return keypoints
    
    def keypoints_to_graph(self, keypoints, image_width, image_height):
        # keypoints is expected to be a tensor with shape (num_keypoints, 4),
        # where each keypoint is (x, y, score, label).
        # Convert all elements in keypoints to tensors if they are not already
        keypoints = [torch.tensor(kp, dtype=torch.float32).to(device) if not isinstance(kp, torch.Tensor) else kp for kp in keypoints]

        # Then stack them
        keypoints = torch.stack(keypoints).to(device)        
        
        # Remove duplicates: Only keep the keypoint with the highest score for each label
        unique_labels, best_keypoint_indices = torch.unique(keypoints[:, 3], return_inverse=True)
        best_scores, best_indices = torch.max(keypoints[:, 2].unsqueeze(0) * (best_keypoint_indices == torch.arange(len(unique_labels)).unsqueeze(1).cuda()), dim=1)
        keypoints = keypoints[best_indices]
        
        print("init keypoints in graph features", keypoints)

        # Normalize x and y to be in the range [-1, 1]
        keypoints[:, 0] = (keypoints[:, 0] - image_width / 2) / (image_width / 2)
        keypoints[:, 1] = (keypoints[:, 1] - image_height / 2) / (image_height / 2)

        # Use only x, y, and score for the graph features
        graph_features = keypoints[:, :4]  # Now shape is (num_keypoints, 3)
        
        # Ensure the shape is [num_keypoints, 3] before returning
        graph_features = graph_features.view(-1, 4)  # Reshape to ensure it's [num_keypoints, 3]
        print("graph features", graph_features)
        print("graph features shape", graph_features.shape)

        return graph_features
        
    def forward(self, img):
        img = img.unsqueeze(0).to(device)
        # Temporarily set the keypoint model to evaluation mode
        keypoint_model_training = self.keypoint_model.training  # Save the current mode
        self.keypoint_model.eval()
        with torch.no_grad():
            output = self.keypoint_model(img)
        # Set the keypoint model back to its previous mode
        self.keypoint_model.train(keypoint_model_training)
        img = (img[0].permute(1,2,0).detach().cpu().numpy() * 255).astype(np.uint8)
        labeled_keypoints = self.process_model_output(output)
        # Initialize x with zeros for 6 nodes with 3 features each
        x = torch.zeros(1, 6, 4, device=device)

        # Generate graph input tensor
        keypoints = self.keypoints_to_graph(labeled_keypoints, 640, 480)

        # Ensure that keypoints are on the correct device and fill in x
        num_keypoints_detected = keypoints.size(0)
        if num_keypoints_detected <= 6:
            x[:, :num_keypoints_detected, :] = keypoints.to(device)
        else:
            raise ValueError("Number of keypoints detected exceeds the maximum of 6.")
        print("X", x)
        logits = self.encoder(x,self.rel_rec,self.rel_send)
        edges = my_softmax(logits, -1).to(device)
        KGNN2D = self.decoder(x, edges, self.rel_rec, self.rel_send,0)# args.prediction_steps)
#         print("edges before softmax", logits)
#         print("edges after softmax", edges)
#         print("rel rec", self.rel_rec)
#         print("rel send", self.rel_send)
        
        return logits,KGNN2D,labeled_keypoints


In [83]:
# def loss_edges(valid_points, edges):
#     off_diag = np.ones([6, 6]) - np.eye(6)
#     idx =  torch.LongTensor(np.where(off_diag)[1].reshape(6,5)).cuda()
#     if valid_points.ndim == 1:
#         valid_points = valid_points.unsqueeze(0)  # Reshape to 2D if necessary

#     relations = torch.zeros(valid_points.shape[0],valid_points.shape[1]*(valid_points.shape[1]-1)).cuda()
#     for count,vis in enumerate(valid_points):
#         vis = vis.view(-1,1) 
#         vis = vis*vis.t()
#         vis = torch.gather(vis,1,idx)
#         relations[count] = vis.view(-1)
#     relations = relations.type(torch.LongTensor).cuda() 
#     loss_edges = func.cross_entropy(edges.view(-1, 2), relations.view(-1))
#     return loss_edges

# def loss_kp(gt_keypoints, pred_keypoints):
#     # Convert pred_keypoints to tensor if it's a list
    
#     if isinstance(pred_keypoints, list):
#         pred_keypoints = torch.stack([torch.tensor(kp, device=gt_keypoints.device, dtype=torch.float32) if isinstance(kp, list) else kp for kp in pred_keypoints])

#     # Ensure gt_keypoints is a tensor
#     if not isinstance(gt_keypoints, torch.Tensor):
#         gt_keypoints = torch.tensor(gt_keypoints, dtype=torch.float32, device=pred_keypoints.device)

#     # Check if the shape of gt_keypoints is as expected, it should be [N, M] where N is the number of keypoints and M is the properties of each keypoint (like x, y, visibility, etc.)
#     if gt_keypoints.dim() != 2 or gt_keypoints.size(-1) < 3:
#         raise ValueError("gt_keypoints must be a 2D tensor with shape [N, M] where M >= 3.")

#     # Initialize a mask for selecting valid keypoints in gt_keypoints
#     valid_gt_mask = (gt_keypoints[:, -1] == 1)

#     # Ensure the mask is one-dimensional
#     if valid_gt_mask.dim() != 1:
#         raise ValueError("The mask must be one-dimensional")

#     # Filter the gt_keypoints and pred_keypoints based on the mask
#     filtered_gt_keypoints = gt_keypoints[valid_gt_mask][:, :2]  # x, y columns
#     filtered_pred_keypoints = pred_keypoints[valid_gt_mask][:, :2]  # x, y columns

#     # Compute the loss using Smooth L1 Loss on the filtered keypoints
#     loss = func.smooth_l1_loss(filtered_pred_keypoints, filtered_gt_keypoints, reduction='none')

#     # Apply the mask to the loss to consider only valid keypoints
#     valid_loss = loss[valid_gt_mask]
#     return valid_loss.sum() / valid_gt_mask.float().sum()  
    
# def kgnn2d_loss(gt_keypoints, pred_keypoints):
#     loss = func.mse_loss(pred_keypoints, gt_keypoints)
    
#     return loss


In [84]:
def encode_onehot(labels):
    classes = set(labels)
    classes_dict = {c: np.identity(len(classes))[i, :] for i, c in
                    enumerate(classes)}
    labels_onehot = np.array(list(map(classes_dict.get, labels)),
                             dtype=np.int32)
    return labels_onehot
def process_keypoints(keypoints):
    # Assuming keypoints is a list of Nx3 tensors where N is the number of keypoints
    # and each keypoint is represented as [x, y, visibility]
    # Remove the unnecessary middle dimension
    keypoints = [kp.squeeze(1) for kp in keypoints]
    visibilities = [kp[:, 2] for kp in keypoints]  # Extract visibility flags
    valid_vis_all = torch.cat([v == 1 for v in visibilities]).long().cuda()
    valid_invis_all = torch.cat([v == 0 for v in visibilities]).long().cuda()

    keypoints_gt = torch.cat([kp[:, :2] for kp in keypoints]).float().cuda()  # Gather all keypoints and discard visibility flags
    keypoints_gt = keypoints_gt.view(-1, 2).unsqueeze(0)  # Add an extra dimension to match expected shape for loss_edges

    return keypoints_gt, valid_vis_all, valid_invis_all

In [85]:
# import os
# import torch
# import torchvision
# import numpy as np
# from torchvision.transforms import functional as F
# from PIL import Image
# import cv2

# model = KeypointPipeline(weights_path)
# model = model.to(device)

# new_weights_path = '/home/jc-merlab/Pictures/Data/trained_models/krcnn_occ_b32_e200_v0.pth'
# model = torch.load(weights_path).to(device)

# # image = Image.open("/home/jc-merlab/Pictures/Data/split_folder_output-2023-10-22/test/images/002510.rgb.jpg")
# image = Image.open("/home/jc-merlab/Pictures/Data/2023-08-14-Occluded/000207.rgb.jpg")
# print(type(image))

# image = F.to_tensor(image).to(device)
# # image.unsqueeze_(0)
# print(image.shape)
# # image = list(image)

# with torch.no_grad():
#     model.to(device)
#     model.eval()
#     output = model(image)
    
# print(output[1])
    
# pred_kp = output[1]

# # Replace these with your actual image dimensions
# image_width = 640
# image_height = 480


# # Denormalize keypoints
# # denormalized_x = (pred_kp[:, :, 0] * (image_width / 2)) + (image_width / 2)
# # denormalized_y = (pred_kp[:, :, 1] * (image_height / 2)) + (image_height / 2)

# # Denormalize keypoints
# denormalized_x = (pred_kp[:, :, 0] + 1) * (image_width / 2)
# denormalized_y = (pred_kp[:, :, 1] + 1) * (image_height / 2)

# # Stack the denormalized x and y coordinates together to form [n, 2] tensor
# denormalized_keypoints = torch.stack((denormalized_x, denormalized_y), dim=2)

# print("Denormalized Keypoints:", denormalized_keypoints)

# kp_numpy = denormalized_keypoints.cpu().numpy()
# kp_flat_list = kp_numpy.reshape(-1).tolist()  # Convert to a flat list


In [86]:
# def visualize_keypoints(image_path, all_keypoints, point_radius=5, keypoint_color=(255, 0, 0)):
#     """
#     Visualize keypoints on the given image. Expects keypoints in the format of a list with a flat structure [x1,y1,x2,y2,...]
#     for each keypoint set associated with an object.
#     """
#     image = cv2.imread(image_path)
    
# #     if bboxes:
# #         for bbox in bboxes:
# #             cv2.rectangle(image, (bbox[0], bbox[1]), (bbox[2], bbox[3]), bbox_color, 2)
    
#     for keypoints in all_keypoints:
#         for i in range(0, len(keypoints), 2):
#             x, y = int(keypoints[i]), int(keypoints[i+1])
#             cv2.circle(image, (x, y), point_radius, keypoint_color, -1)
    
#     return image

# image = "/home/jc-merlab/Pictures/Data/2023-08-14-Occluded/000207.rgb.jpg"

# output_image = visualize_keypoints(image,[kp_flat_list])

# # Display or save the output image as needed
# cv2.imshow("Keypoints", output_image)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

In [87]:
import os
import json
import torch
import torchvision
import numpy as np
from torchvision.transforms import functional as F
from PIL import Image
import cv2

def calculate_accuracy(pred_kps, gt_kps, threshold=10):
    """Calculate accuracy based on distance threshold."""
    distances = torch.norm(pred_kps - gt_kps, dim=2)
    correct = torch.le(distances, threshold).all(dim=1)
    accuracy = torch.mean(correct.float())
    return accuracy.item()

def visualize_keypoints(image, pred_keypoints, gt_keypoints, pred_color=(0, 255, 0), gt_color=(255, 0, 0), pred_radius=7, gt_radius=5):
    """Visualize predicted and ground truth keypoints."""
    for x, y in pred_keypoints:
        cv2.circle(image, (int(x), int(y)), pred_radius, pred_color, -1)
    for x, y in gt_keypoints:
        cv2.circle(image, (int(x), int(y)), gt_radius, gt_color, -1)
    return image

In [88]:
weights_path = '/home/jc-merlab/Pictures/Data/trained_models/keypointsrcnn_weights_sim_b1_e25_v0.pth'
# new_weights_path = '/home/jc-merlab/Pictures/Data/trained_models/checkpoints/occ_net__epoch_150.pth'
new_weights_path = '/home/jc-merlab/Pictures/Data/trained_models/krcnn_occ_b8_e100_v0.pth'
# checkpoint = torch.load(new_weights_path)
# Load your model
model = KeypointPipeline(weights_path).to(device)
# model.load_state_dict(checkpoint['model_state'])
model = torch.load(new_weights_path).to(device)
model.eval()

source_folder = '/home/jc-merlab/Pictures/Data/only_occ_data/'
output_folder = '/home/jc-merlab/Pictures/Data/occ_results/8/'  # Folder to save visualized images

if not os.path.exists(output_folder):
    os.makedirs(output_folder)

total_accuracy = 0
num_images = 0
image_width = 640
image_height = 480

for filename in os.listdir(source_folder):
    if filename.endswith(".rgb.jpg"):
        image_path = os.path.join(source_folder, filename)
        annotation_path = os.path.join(source_folder, filename.replace('.rgb.jpg','.json'))

        # Load image and annotation
        image = Image.open(image_path)
        with open(annotation_path, 'r') as f:
            annotation = json.load(f)

        # Preprocess image
        input_tensor = F.to_tensor(image).to(device)

        # Predict keypoints
        with torch.no_grad():
            output = model(input_tensor)
        pred_kps = output[1]

        # Denormalize keypoints
        denormalized_pred_kps = torch.stack(((pred_kps[:, :, 0] + 1) * (image.width / 2), 
                                             (pred_kps[:, :, 1] + 1) * (image.height / 2)), dim=2)
        
#         denormalized_pred_kps = torch.stack(((pred_kps[:, :, 0] * (image_width / 2)) + (image.width / 2), 
#                                              (pred_kps[:, :, 1] * (image_height / 2)) + (image.height / 2)), dim=2)
        
#         denormalized_x = (pred_kp[:, :, 0] * (image_width / 2)) + (image_width / 2)
# # denormalized_y = (pred_kp[:, :, 1] * (image_height / 2)) + (image_height / 2)


        # Extract ground truth keypoints and format them correctly
        gt_kps = torch.tensor([kp[0][:2] for kp in annotation['keypoints']], dtype=torch.float32)

        # Calculate accuracy
        accuracy = calculate_accuracy(denormalized_pred_kps.to(device), gt_kps.unsqueeze(0).to(device))
        total_accuracy += accuracy
        num_images += 1

        # Visualize keypoints
        image_np = np.array(image)
        output_image = visualize_keypoints(image_np, denormalized_pred_kps[0].cpu().numpy(), gt_kps.numpy())

        print(f'Accuracy for {filename}: {accuracy * 100:.2f}%')
        
        # Save the output image
        output_image_path = os.path.join(output_folder, f'visualized_{filename}')
        cv2.imwrite(output_image_path, cv2.cvtColor(output_image, cv2.COLOR_RGB2BGR))
        
# Calculate mean accuracy
if num_images > 0:
    mean_accuracy = total_accuracy / num_images
    print(f'Mean Accuracy across all images: {mean_accuracy * 100:.2f}%')
else:
    print('No images were processed.')

Using MLP graph encoder.
Using learned graph decoder.
init keypoints in graph features tensor([[258.0000, 367.0000,   0.9999,   1.0000],
        [258.0000, 283.0000,   0.9999,   2.0000],
        [184.0000, 255.0000,   0.9995,   3.0000],
        [190.0000, 235.0000,   0.9997,   4.0000],
        [ 92.0000, 205.0000,   0.9959,   5.0000],
        [ 75.0000, 197.0000,   0.9917,   6.0000]], device='cuda:0')
graph features tensor([[-0.1938,  0.5292,  0.9999,  1.0000],
        [-0.1938,  0.1792,  0.9999,  2.0000],
        [-0.4250,  0.0625,  0.9995,  3.0000],
        [-0.4062, -0.0208,  0.9997,  4.0000],
        [-0.7125, -0.1458,  0.9959,  5.0000],
        [-0.7656, -0.1792,  0.9917,  6.0000]], device='cuda:0')
graph features shape torch.Size([6, 4])
X tensor([[[-0.1938,  0.5292,  0.9999,  1.0000],
         [-0.1938,  0.1792,  0.9999,  2.0000],
         [-0.4250,  0.0625,  0.9995,  3.0000],
         [-0.4062, -0.0208,  0.9997,  4.0000],
         [-0.7125, -0.1458,  0.9959,  5.0000],
         

init keypoints in graph features tensor([[258.0000, 367.0000,   1.0000,   1.0000],
        [258.0000, 283.0000,   0.9999,   2.0000],
        [179.0000, 298.0000,   0.9992,   3.0000],
        [175.0000, 278.0000,   0.9997,   4.0000],
        [ 91.0000, 226.0000,   0.9990,   5.0000],
        [ 84.0000, 206.0000,   0.9941,   6.0000]], device='cuda:0')
graph features tensor([[-0.1938,  0.5292,  1.0000,  1.0000],
        [-0.1938,  0.1792,  0.9999,  2.0000],
        [-0.4406,  0.2417,  0.9992,  3.0000],
        [-0.4531,  0.1583,  0.9997,  4.0000],
        [-0.7156, -0.0583,  0.9990,  5.0000],
        [-0.7375, -0.1417,  0.9941,  6.0000]], device='cuda:0')
graph features shape torch.Size([6, 4])
X tensor([[[-0.1938,  0.5292,  1.0000,  1.0000],
         [-0.1938,  0.1792,  0.9999,  2.0000],
         [-0.4406,  0.2417,  0.9992,  3.0000],
         [-0.4531,  0.1583,  0.9997,  4.0000],
         [-0.7156, -0.0583,  0.9990,  5.0000],
         [-0.7375, -0.1417,  0.9941,  6.0000]]], device='cuda:0

init keypoints in graph features tensor([[258.0000, 283.0000,   0.9999,   2.0000],
        [283.0000, 207.0000,   0.9997,   3.0000],
        [302.0000, 214.0000,   0.9998,   4.0000],
        [358.0000, 130.0000,   0.8431,   5.0000]], device='cuda:0')
graph features tensor([[-0.1938,  0.1792,  0.9999,  2.0000],
        [-0.1156, -0.1375,  0.9997,  3.0000],
        [-0.0563, -0.1083,  0.9998,  4.0000],
        [ 0.1187, -0.4583,  0.8431,  5.0000]], device='cuda:0')
graph features shape torch.Size([4, 4])
X tensor([[[-0.1938,  0.1792,  0.9999,  2.0000],
         [-0.1156, -0.1375,  0.9997,  3.0000],
         [-0.0563, -0.1083,  0.9998,  4.0000],
         [ 0.1187, -0.4583,  0.8431,  5.0000],
         [ 0.0000,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  0.0000]]], device='cuda:0')
Accuracy for 002400.rgb.jpg: 0.00%
init keypoints in graph features tensor([[258.0000, 367.0000,   1.0000,   1.0000],
        [257.0000, 283.0000,   0.9998,   2.0000],
        [304.0000, 21

init keypoints in graph features tensor([[258.0000, 367.0000,   1.0000,   1.0000],
        [258.0000, 283.0000,   0.9999,   2.0000],
        [310.0000, 223.0000,   0.9991,   3.0000],
        [326.0000, 237.0000,   0.9998,   4.0000]], device='cuda:0')
graph features tensor([[-0.1938,  0.5292,  1.0000,  1.0000],
        [-0.1938,  0.1792,  0.9999,  2.0000],
        [-0.0312, -0.0708,  0.9991,  3.0000],
        [ 0.0188, -0.0125,  0.9998,  4.0000]], device='cuda:0')
graph features shape torch.Size([4, 4])
X tensor([[[-0.1938,  0.5292,  1.0000,  1.0000],
         [-0.1938,  0.1792,  0.9999,  2.0000],
         [-0.0312, -0.0708,  0.9991,  3.0000],
         [ 0.0188, -0.0125,  0.9998,  4.0000],
         [ 0.0000,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  0.0000]]], device='cuda:0')
Accuracy for 002554.rgb.jpg: 0.00%
init keypoints in graph features tensor([[258.0000, 367.0000,   0.9999,   1.0000],
        [258.0000, 283.0000,   1.0000,   2.0000],
        [178.0000, 27

init keypoints in graph features tensor([[258.0000, 367.0000,   0.9999,   1.0000],
        [257.0000, 282.0000,   0.9999,   2.0000],
        [193.0000, 234.0000,   0.9999,   3.0000],
        [203.0000, 217.0000,   0.9997,   4.0000],
        [187.0000, 121.0000,   0.9980,   5.0000],
        [199.0000, 102.0000,   0.9940,   6.0000]], device='cuda:0')
graph features tensor([[-0.1938,  0.5292,  0.9999,  1.0000],
        [-0.1969,  0.1750,  0.9999,  2.0000],
        [-0.3969, -0.0250,  0.9999,  3.0000],
        [-0.3656, -0.0958,  0.9997,  4.0000],
        [-0.4156, -0.4958,  0.9980,  5.0000],
        [-0.3781, -0.5750,  0.9940,  6.0000]], device='cuda:0')
graph features shape torch.Size([6, 4])
X tensor([[[-0.1938,  0.5292,  0.9999,  1.0000],
         [-0.1969,  0.1750,  0.9999,  2.0000],
         [-0.3969, -0.0250,  0.9999,  3.0000],
         [-0.3656, -0.0958,  0.9997,  4.0000],
         [-0.4156, -0.4958,  0.9980,  5.0000],
         [-0.3781, -0.5750,  0.9940,  6.0000]]], device='cuda:0

...

init keypoints in graph features tensor([[258.0000, 283.0000,   0.9999,   2.0000],
        [232.0000, 207.0000,   0.9998,   3.0000],
        [252.0000, 201.0000,   0.9996,   4.0000],
        [237.0000, 103.0000,   0.9587,   5.0000]], device='cuda:0')
graph features tensor([[-0.1938,  0.1792,  0.9999,  2.0000],
        [-0.2750, -0.1375,  0.9998,  3.0000],
        [-0.2125, -0.1625,  0.9996,  4.0000],
        [-0.2594, -0.5708,  0.9587,  5.0000]], device='cuda:0')
graph features shape torch.Size([4, 4])
X tensor([[[-0.1938,  0.1792,  0.9999,  2.0000],
         [-0.2750, -0.1375,  0.9998,  3.0000],
         [-0.2125, -0.1625,  0.9996,  4.0000],
         [-0.2594, -0.5708,  0.9587,  5.0000],
         [ 0.0000,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  0.0000]]], device='cuda:0')
Accuracy for 001948.rgb.jpg: 0.00%
init keypoints in graph features tensor([[260.0000, 366.0000,   0.9921,   1.0000],
        [258.0000, 203.0000,   0.9995,   3.0000],
        [279.0000, 20

init keypoints in graph features tensor([[258.0000, 367.0000,   1.0000,   1.0000],
        [258.0000, 283.0000,   1.0000,   2.0000],
        [281.0000, 208.0000,   0.9961,   3.0000],
        [356.0000, 130.0000,   0.9976,   5.0000],
        [362.0000, 109.0000,   0.9981,   6.0000]], device='cuda:0')
graph features tensor([[-0.1938,  0.5292,  1.0000,  1.0000],
        [-0.1938,  0.1792,  1.0000,  2.0000],
        [-0.1219, -0.1333,  0.9961,  3.0000],
        [ 0.1125, -0.4583,  0.9976,  5.0000],
        [ 0.1313, -0.5458,  0.9981,  6.0000]], device='cuda:0')
graph features shape torch.Size([5, 4])
X tensor([[[-0.1938,  0.5292,  1.0000,  1.0000],
         [-0.1938,  0.1792,  1.0000,  2.0000],
         [-0.1219, -0.1333,  0.9961,  3.0000],
         [ 0.1125, -0.4583,  0.9976,  5.0000],
         [ 0.1313, -0.5458,  0.9981,  6.0000],
         [ 0.0000,  0.0000,  0.0000,  0.0000]]], device='cuda:0')
Accuracy for 002196.rgb.jpg: 0.00%
init keypoints in graph features tensor([[258.0000, 367.00

init keypoints in graph features tensor([[258.0000, 366.0000,   0.9999,   1.0000],
        [260.0000, 284.0000,   0.9594,   2.0000],
        [310.0000, 222.0000,   0.9987,   3.0000],
        [326.0000, 236.0000,   0.9999,   4.0000],
        [374.0000, 322.0000,   0.9959,   5.0000],
        [358.0000, 327.0000,   0.9952,   6.0000]], device='cuda:0')
graph features tensor([[-0.1938,  0.5250,  0.9999,  1.0000],
        [-0.1875,  0.1833,  0.9594,  2.0000],
        [-0.0312, -0.0750,  0.9987,  3.0000],
        [ 0.0188, -0.0167,  0.9999,  4.0000],
        [ 0.1688,  0.3417,  0.9959,  5.0000],
        [ 0.1187,  0.3625,  0.9952,  6.0000]], device='cuda:0')
graph features shape torch.Size([6, 4])
X tensor([[[-0.1938,  0.5250,  0.9999,  1.0000],
         [-0.1875,  0.1833,  0.9594,  2.0000],
         [-0.0312, -0.0750,  0.9987,  3.0000],
         [ 0.0188, -0.0167,  0.9999,  4.0000],
         [ 0.1688,  0.3417,  0.9959,  5.0000],
         [ 0.1187,  0.3625,  0.9952,  6.0000]]], device='cuda:0

init keypoints in graph features tensor([[258.0000, 367.0000,   1.0000,   1.0000],
        [258.0000, 283.0000,   0.9999,   2.0000],
        [ 76.0000, 277.0000,   0.9978,   5.0000],
        [ 74.0000, 257.0000,   0.9828,   6.0000]], device='cuda:0')
graph features tensor([[-0.1938,  0.5292,  1.0000,  1.0000],
        [-0.1938,  0.1792,  0.9999,  2.0000],
        [-0.7625,  0.1542,  0.9978,  5.0000],
        [-0.7688,  0.0708,  0.9828,  6.0000]], device='cuda:0')
graph features shape torch.Size([4, 4])
X tensor([[[-0.1938,  0.5292,  1.0000,  1.0000],
         [-0.1938,  0.1792,  0.9999,  2.0000],
         [-0.7625,  0.1542,  0.9978,  5.0000],
         [-0.7688,  0.0708,  0.9828,  6.0000],
         [ 0.0000,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  0.0000]]], device='cuda:0')
Accuracy for 001438.rgb.jpg: 0.00%
init keypoints in graph features tensor([[258.0000, 367.0000,   0.9999,   1.0000],
        [258.0000, 283.0000,   0.9999,   2.0000],
        [179.0000, 29

init keypoints in graph features tensor([[258.0000, 367.0000,   0.9999,   1.0000],
        [258.0000, 283.0000,   0.9999,   2.0000],
        [194.0000, 235.0000,   0.9998,   3.0000],
        [207.0000, 217.0000,   0.9999,   4.0000]], device='cuda:0')
graph features tensor([[-0.1938,  0.5292,  0.9999,  1.0000],
        [-0.1938,  0.1792,  0.9999,  2.0000],
        [-0.3938, -0.0208,  0.9998,  3.0000],
        [-0.3531, -0.0958,  0.9999,  4.0000]], device='cuda:0')
graph features shape torch.Size([4, 4])
X tensor([[[-0.1938,  0.5292,  0.9999,  1.0000],
         [-0.1938,  0.1792,  0.9999,  2.0000],
         [-0.3938, -0.0208,  0.9998,  3.0000],
         [-0.3531, -0.0958,  0.9999,  4.0000],
         [ 0.0000,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  0.0000]]], device='cuda:0')
Accuracy for 001764.rgb.jpg: 0.00%
Mean Accuracy across all images: 38.69%
